<a href="https://colab.research.google.com/github/DJlee331/Julia_Practice_DJ/blob/main/BVAR_JUlia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
using Pkg
Pkg.add("FredData")
Pkg.add("Dates")
Pkg.add("Distributions")

    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
   Installed TZJData ─── v1.5.0+2025b
   Installed Mocking ─── v0.8.1
   Installed TimeZones ─ v1.21.3
   Installed FredData ── v0.6.0
    Updating `~/.julia/environments/v1.10/Project.toml`
  [6e85b7ee] + FredData v0.6.0
    Updating `~/.julia/environments/v1.10/Manifest.toml`
  [6e85b7ee] + FredData v0.6.0
  [78c3b35d] + Mocking v0.8.1
  [dc5dba14] + TZJData v1.5.0+2025b
  [f269a46b] + TimeZones v1.21.3
Precompiling packages...
   3759.8 ms  ✓ TZJData
   3153.6 ms  ✓ Mocking
   9129.1 ms  ✓ TimeZones
    883.0 ms  ✓ TimeZones → TimeZonesRecipesBaseExt
   2798.3 ms  ✓ FredData
  5 dependencies successfully precompiled in 25 seconds. 455 already precompiled.
   Resolving package versions...
    Updating `~/.julia/environments/v1.10/Project.toml`
  [ade2ca70] + Dates
  No Changes to `~/.julia/environments/v1.10/Manifest.toml`
   Resolving package versions...
    Updating `~/.julia/environments/v

In [ ]:
using DataFrames, CSV
using Statistics, LinearAlgebra
using Random, Distributions
Random.seed!(4412)

TaskLocalRNG()

In [ ]:
using FredData
using Dates

In [ ]:
fred = FredData.Fred("adeee66f40c0687c4decaba2442fcaff") # Replace YOUR_FRED_API_KEY with your key

FRED API Connection
	url: https://api.stlouisfed.org/fred/
	key: adeee66f40c0687c4decaba2442fcaff


In [ ]:
# 2. 시리즈 ID로 데이터 요청 (예: GDP)
DF = get_data(fred, "PCEPI").data[:,[:date,:value]]; # PCE
rename!(DF,:value => :pce);
series = get_data(fred, "PCEPILFE").data[:,[:date,:value]]; # PCE core
rename!(series,:value => :pcecore);
DF = outerjoin(DF,series, on = :date);
series = get_data(fred, "CPIAUCSL").data[:,[:date,:value]]; # CPI
rename!(series,:value => :cpi);
DF = outerjoin(DF,series, on = :date);
series = get_data(fred, "CPILFESL").data[:,[:date,:value]]; # CPI Core
rename!(series,:value => :cpicore);
DF = outerjoin(DF,series, on = :date);
#series = get_data(fred, "T10YIE").data[:,[:date,:value]]; # 10year BEI
#rename!(series,:value => :bei);
#DF = outerjoin(DF,series, on = :date);
series = get_data(fred, "INDPRO").data[:,[:date,:value]]; # Industrial Production
rename!(series,:value => :indpro);
DF = outerjoin(DF,series, on = :date);
series = get_data(fred, "UNRATE").data[:,[:date,:value]]; # Unemployment Rate
rename!(series,:value => :unemp);
DF = outerjoin(DF,series, on = :date);
series = get_data(fred, "PAYEMS").data[:,[:date,:value]]; # Total NonFarm employees
rename!(series,:value => :employ);
DF = outerjoin(DF,series, on = :date);
series = get_data(fred, "DFF").data[:,[:date,:value]]; # Federal funds effective rate
rename!(series,:value => :ffr);
DF = outerjoin(DF,series, on = :date);
series = get_data(fred, "DGS10").data[:,[:date,:value]]; # Market yield on Us Treasury 10y
rename!(series,:value => :year10);
DF = outerjoin(DF,series, on = :date);
series = get_data(fred, "DAAA").data[:,[:date,:value]]; # Moody's Seasoned Aaa Corporate bond yield
rename!(series,:value => :corpbond);
DF = outerjoin(DF,series, on = :date);
#series = get_data(fred, "MSPUS").data[:,[:date,:value]]; # Median Sales Price of Houses Sold for the United States
#rename!(series,:value => :hpi);
#DF = outerjoin(DF,series, on = :date);
series = get_data(fred, "TOTALSA").data[:,[:date,:value]]; # Total Vehicle Sales
rename!(series,:value => :vehicle);
DF = outerjoin(DF,series, on = :date);
series = get_data(fred, "WTISPLC").data[:,[:date,:value]]; # Spot Crude Oil Price: West Texas Intermediate (WTI)
rename!(series,:value => :oil);
DF = outerjoin(DF,series, on = :date);

┌ Warning: Metadata 'notes' not returned from server.
└ @ FredData ~/.julia/packages/FredData/5M7x4/src/get_data.jl:77


In [ ]:
sort!(DF, :date);
DF.month = Date.(year.(DF.date),month.(DF.date),1);
df_m = combine(groupby(DF, :month),
    names(DF, Not([:date, :month])) .=> (x -> mean(filter(!isnan, skipmissing(x)))) .=> names(DF, Not([:date,:month]))
);
rename!(df_m,:month =>:date);
start_date = Date("1984-01-01")
end_date   = Date("2019-12-01")

DF_F = df_m[(df_m.date .>= start_date) .& (df_m.date .<= end_date), :];
DF_F[[1:5; end-4:end],:]

Row,date,pce,pcecore,cpi,cpicore,indpro,unemp,employ,ffr,year10,corpbond,vehicle,oil
,Date,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,1984-01-01,47.564,47.121,102.1,102.5,53.0088,8.0,92673.0,9.55839,11.6743,12.2,14.21,29.69
2,1984-02-01,47.884,47.418,102.6,102.8,53.2486,7.8,93157.0,9.58759,11.8421,12.0835,14.381,30.145
3,1984-03-01,48.041,47.598,102.9,103.2,53.5028,7.8,93429.0,9.91323,12.3191,12.5736,14.21,30.761
4,1984-04-01,48.224,47.822,103.3,103.7,53.8233,7.7,93792.0,10.291,12.6335,12.8085,14.357,30.618
5,1984-05-01,48.285,47.933,103.5,104.1,54.1104,7.4,94098.0,10.3194,13.4086,13.2827,14.856,30.52
6,2019-08-01,103.67,103.797,256.036,263.877,102.781,3.6,151171.0,2.12581,1.62636,2.97727,17.882,54.81
7,2019-09-01,103.733,103.873,256.43,264.388,102.46,3.5,151365.0,2.043,1.6995,3.032,17.72,56.95
8,2019-10-01,103.917,104.028,257.155,264.97,101.588,3.6,151460.0,1.82968,1.70682,3.00818,17.049,53.96
9,2019-11-01,104.024,104.092,257.879,265.548,102.149,3.6,151668.0,1.55333,1.81211,3.05737,17.549,57.03


In [ ]:
log_level = vec([1 2 3 4 5 7  12]);
# names(DF_F,Not(:date))[log_level]
transform!(DF_F, names(DF_F,Not(:date))[log_level] .=> ByRow(x -> log(x) * 100) .=> names(DF_F,Not(:date))[log_level] );
DF_F[[1:5; end-4:end],:]

Row,date,pce,pcecore,cpi,cpicore,indpro,unemp,employ,ffr,year10,corpbond,vehicle,oil
,Date,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,1984-01-01,595.638,595.395,613.685,613.77,598.405,8.0,704.201,9.55839,11.6743,12.2,14.21,582.624
2,1984-02-01,595.811,595.558,613.791,613.833,598.519,7.8,704.246,9.58759,11.8421,12.0835,14.381,583.071
3,1984-03-01,595.896,595.656,613.854,613.917,598.639,7.8,704.272,9.91323,12.3191,12.5736,14.21,583.664
4,1984-04-01,595.994,595.777,613.938,614.021,598.788,7.7,704.306,10.291,12.6335,12.8085,14.357,583.527
5,1984-05-01,596.026,595.837,613.979,614.104,598.922,7.4,704.334,10.3194,13.4086,13.2827,14.856,583.434
6,2019-08-01,614.015,614.041,631.812,632.355,613.829,3.6,708.391,2.12581,1.62636,2.97727,17.882,599.243
7,2019-09-01,614.028,614.057,631.84,632.39,613.761,3.5,708.401,2.043,1.6995,3.032,17.72,600.195
8,2019-10-01,614.066,614.089,631.891,632.429,613.576,3.6,708.407,1.82968,1.70682,3.00818,17.049,598.852
9,2019-11-01,614.088,614.102,631.942,632.468,613.696,3.6,708.418,1.55333,1.81211,3.05737,17.549,600.23


In [ ]:
DF_Q =copy(DF_F);
DF_Q.quarter = Date.(year.(DF_F.date),quarterofyear.(Date.(DF_F.date)).*3 ,1); # 분기 말월 1일 기준
DF_Q = combine(groupby(DF_Q, :quarter),names(DF_Q, Not([:date, :quarter])) .=> mean)
rename!(DF_Q, :quarter => :date);
DF_Q[[1:4;end-3:end],:]

Row,date,pce_mean,pcecore_mean,cpi_mean,cpicore_mean,indpro_mean,unemp_mean,employ_mean,ffr_mean,year10_mean,corpbond_mean,vehicle_mean,oil_mean
,Date,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,1984-03-01,386.764,385.817,463.018,463.311,397.505,7.86667,1144.13,9.6864,11.9452,12.2857,14.267,340.769
2,1984-06-01,387.729,386.982,463.957,464.535,399.047,7.43333,1145.24,10.5573,13.2017,13.2134,14.6287,341.336
3,1984-09-01,388.497,387.916,464.823,465.776,399.73,7.43333,1146.22,11.3913,12.867,12.9896,14.332,337.085
4,1984-12-01,389.112,388.625,465.681,466.783,399.778,7.3,1147.11,9.26425,11.7442,12.3506,14.6727,331.025
5,2019-03-01,463.328,463.438,553.491,556.518,463.479,3.86667,1191.93,2.40172,2.65369,3.83095,17.258,400.29
6,2019-06-01,463.886,463.859,554.223,556.956,462.9,3.63333,1192.29,2.39743,2.33379,3.59662,17.572,408.864
7,2019-09-01,464.126,464.23,554.553,557.537,462.904,3.6,1192.61,2.19057,1.79498,3.10082,17.7613,403.174
8,2019-12-01,464.518,464.581,555.252,558.166,462.392,3.6,1192.93,1.64466,1.79393,3.02423,17.363,404.139


In [ ]:
Pkg.add("Interpolations")

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.10/Project.toml`
  No Changes to `~/.julia/environments/v1.10/Manifest.toml`


In [ ]:
using Interpolations

In [ ]:
# 각 월의 일 수 차이로 변환 후 내삽
Y = Matrix(DF_Q[:,2:end]);
x_monthly = Dates.value.(DF_F.date .- DF_F.date[1]);
x_quarterly = Dates.value.(DF_Q.date .- DF_Q.date[1]);
results = map(i -> extrapolate(interpolate((x_quarterly,), Y[:, i], Gridded(Linear())), Line()).(x_monthly), 1:size(Y, 2));
results = hcat(results...);

DF_M_Interpolated = DataFrame(results, names(DF_Q,Not(:date)))  # 자동으로 x1, x2, ..., x12 라벨 부여
DF_M_Interpolated.date = DF_F.date;
select!(DF_M_Interpolated, :date, :);  # :date 열을 맨 앞으로
DF_M_Interpolated[[1:6;end-5:end],:]

Row,date,pce_mean,pcecore_mean,cpi_mean,cpicore_mean,indpro_mean,unemp_mean,employ_mean,ffr_mean,year10_mean,corpbond_mean,vehicle_mean,oil_mean
,Date,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,1984-01-01,386.764,385.817,463.018,463.311,397.505,7.86667,1144.13,9.6864,11.9452,12.2857,14.267,340.769
2,1984-02-01,387.089,386.209,463.335,463.723,398.025,7.72065,1144.5,9.97987,12.3685,12.5983,14.3889,340.96
3,1984-03-01,387.393,386.577,463.631,464.109,398.51,7.58406,1144.85,10.2544,12.7646,12.8907,14.5029,341.139
4,1984-04-01,387.719,386.969,463.947,464.521,399.03,7.43804,1145.22,10.5479,13.188,13.2033,14.6247,341.329
5,1984-05-01,387.971,387.276,464.23,464.926,399.262,7.43333,1145.55,10.8202,13.0962,13.1429,14.5352,339.996
6,1984-06-01,388.23,387.591,464.522,465.344,399.492,7.43333,1145.88,11.1012,12.9834,13.0674,14.4352,338.563
7,2019-07-01,464.121,464.222,554.545,557.525,462.903,3.60072,1192.6,2.19507,1.8067,3.1116,17.7572,403.298
8,2019-08-01,464.251,464.342,554.776,557.738,462.741,3.6,1192.71,2.0166,1.79465,3.07641,17.6344,403.481
9,2019-09-01,464.385,464.461,555.014,557.952,462.566,3.6,1192.82,1.83063,1.79429,3.05032,17.4987,403.81


Estimate a Bayesian VAR

In [ ]:
# When all Latent variables were observable
Y = Matrix(DF_F[:,Not(([:date]))]);
T,k = size(Y)

(432, 12)

In [ ]:
p=6;

In [ ]:
prod(size(Y))

5184

In [ ]:
function prior_init(Y::Matrix{Float64},p::Int,log_level::Vector{Int},lambda1::Float64=0.09, lambda2::Float64=4.3,lambda3::Float64=1,lambda4::Float64=2.7,lambda5::Float64=4.3,includeconstant::Bool=true)
# Initialized prior(Minnesota prior)
  T,k = size(Y);
  # prior for Innovation Covariance matrix
  S0 = zeros(k,k);
  # Estimate standard errors of AR(1) for each variable
  for i=1:k
    y = Y[:,i];
    epsilon = y[2:end]-hcat(ones(T-1,1), y[1:end-1])*(hcat(ones(T-1,1), y[1:end-1])\y[2:end]);
    S0[i,i] = epsilon'epsilon/T;
  end

  # prior for coefficients
  B0 = zeros(k,k*p);
  B0[log_level,log_level]=1; # For nonstationary variables
  B0[setdiff(1:k,log_level),setdiff(1:k,log_level)]=.5; # For stationary variables
  if includeconstant;
    B0 = hcat(zeros(k,1),B0);
  end
  V0 = ones(prod(size(B0)))

  return B0, V0, ν0, S0, MN_hyper(lambda1,lambda2,lambda3,lambda4,lambda5,delta)
end

In [ ]:
function gibbs_bvar(Y::Matrix, p::Int, n_draws::Int = 2000, burnin::Int = 1000, MN_hyper::MN_hyper,log_level::Matrix{Int})
    T, k = size(Y)
    X, Y_target = make_lag_matrix(Y, p)

    B0, V0, ν0, S0 = make_dummies(n,p,MN_hyper)

    V0_inv = inv(V0)
    B_samples = Vector{Matrix{Float64}}()
    Σ_samples = Vector{Matrix{Float64}}()

    # 초기값
    Σ = Matrix(I, n, n)

    for s in 1:n_draws
        # posterior of B | Σ
        V_post = inv(X'X + inv(V0))                  # k × k
        B_post_mean = V_post * (X'Y_target + inv(V0) * B0)  # k × n
        B_draw = reshape(rand(MvNormal(B_post_mean, Vn)), k, n)

        # residual
        E = Y_target - X * B_draw

        # posterior of Σ | B
        Sn = S0 + E'E
        Σ = inv(rand(Wishart(ν0 + size(Y_target, 1), inv(Sn))))

        push!(B_samples, B_draw)
        push!(Σ_samples, Σ)
    end

    return B_samples, Σ_samples
end

function make_lag_matrix(Y::Matrix, p::Int)
    T, n = size(Y)
    X = hcat([Y[p - i + 1:end - i, :] for i in 1:p]...)  # concaternating lag matrices
    X = hcat(ones(T - p), X)  # 상수항
    Y_target = Y[p+1:end, :]
    return X, Y_target
end

function make_dummies(n::Int,p::Int,lambda1::Float64=0.09, lambda2::Float64=4.3,lambda3::Float64=1,lambda4::Float64=2.7,lambda5::Float64=4.3)
    dummy_obs

    return dummy_obs, MN_hyper(lambda1,lambda2,lambda3,lambda4,lambda5,delta)
end

init_priors (generic function with 1 method)

In [ ]:
struct MN_hyper
    lambda1::Float64       # overall tightness
    lambda2::Float64       # the rate at which the prior variance on higher-order lag coefficients decays
    lambda3::Float64    # Dispersion of the prior on the innovation covariance matrix
    lambda4::Float64    # Sum of coefficient prior
    lambda5::Float64    # the extent to which co-persistence restrictions are imposed on the VAR coefficients
end

LoadError: invalid redefinition of constant Main.MN_hyper

In [ ]:
typeof(log_level)

Matrix{Int64} (alias for Array{Int64, 2})

In [ ]:
function make_dummy_observations(n::Int, p::Int, λ1::Float64, λ3::Float64, ψ::Vector{Float64}, δ::Vector{Float64})
    # Own-lag prior
    ψ =
    Y_d1 = λ1 * Diagonal(ψ .* δ)
    X_d1 = λ1 * hcat(kron(Diagonal(ψ), ones(p)), zeros(n, 1))

    # Sum-of-coefficients prior
    Y_d2 = λ3 * (ψ .* δ)'
    X_d2 = λ3 * hcat(kron(ones(1, p), (ψ .* δ)'), zeros(1, 1))

    # Combine dummy observations
    Y_d = vcat(Y_d1, Y_d2)
    X_d = vcat(X_d1, X_d2)

    return Y_d, X_d
end
